In [597]:
info = {}
info['version'] = !python -V
info['directory'] = !pwd
info['packages'] = !pip list

In [75]:
import pandas as pd
import os
import elsapy
from elsapy.elsdoc import FullDoc, AbsDoc
import json
os.chdir('/Users/tim/nlp_valid')

In [31]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

In [32]:
config

{'apikey': '222a2a95a093a5c6049c145c09ed85a9'}

In [33]:
## Initialize client
client = ElsClient(config['apikey'])

In [37]:
## Scopus (Abtract) document example
# Initialize document with ID as integer
scp_doc = AbsDoc(scp_id = 84872135457)
if scp_doc.read(client):
    print ("scp_doc.title: ", scp_doc.title)
    scp_doc.write()   
else:
    print ("Read document failed.")

scp_doc.title:  Control of somatic tissue differentiation by the long non-coding RNA TINCR


In [721]:
## ScienceDirect (full-text) document example using PII
pii_doc = FullDoc(sd_pii = 'S1674927814000082')
if pii_doc.read(client):
    print ("pii_doc.title: ", pii_doc.title)
    pii_doc.write()   
else:
    print ("Read document failed.")

In [722]:
pii_doc.data

In [60]:
import requests

In [422]:
url = 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=lipitor%20targets&apiKey=222a2a95a093a5c6049c145c09ed85a9'

In [355]:
b = {}
for i in results['pubmed_id']:
    b[str(i)] = requests.get(f'https://api.elsevier.com/content/article/pubmed_id/{i}')

In [543]:

def get_summary(search_string,apikey):
    search_string = search_string.replace(' ','%20')
    url = f'https://api.elsevier.com/content/search/scopus?start=0&count=25&query={search_string}&apiKey={apikey}'
    res = requests.get(url)
    x = json.loads(res.text)
    summary = {}
    try:
        summary['total_results'] = int(x['search-results']['opensearch:totalResults'])
        summary['search_terms'] = x['search-results']['opensearch:Query']['@searchTerms']
    except Exception as e:
        summary['total_results'] = 0
        summary['search_terms'] = search_string
        print(e)
    
    summary['link_used'] = url
    summary['apikey'] = apikey
    return summary

def get_urls(summary):
    #url generation
    search_string = summary['search_terms'].replace(' ','%20')
    apikey = summary['apikey']
    base = f'https://api.elsevier.com/content/search/scopus?start=0&count=25&query={search_string}&apiKey={apikey}'
    x=0
    n=round(summary['total_results']/25)
    urls = []
    for i in range(n+1):
        url = f'https://api.elsevier.com/content/search/scopus?start={x}&count=25&query={search_string}&apiKey={apikey}'
        x+=25
        urls.append(url)
    return(urls)

def get_res(url):
    error_collection=[]
    res = requests.get(url)
    x = json.loads(res.text)
    ids = {}
    try:
        results = pd.DataFrame(x['search-results']['entry'])
        ids = pd.DataFrame(x['search-results']['entry'])['dc:identifier']
        scopus_ids = ",".join(format(i.split(':')[1])for i in ids).split(',')
        results['dc:identifier'] = scopus_ids
        return results
    except Exception as e:
        print (url,e)
        return  pd.DataFrame()
    
def final_result(search_string):
    apikey = '222a2a95a093a5c6049c145c09ed85a9'
    summary = get_summary(search_string,apikey)
    urls = get_urls(summary)
    final_result = []
    for url in urls:
        res = get_res(url)
        final_result.append(res)
    out = pd.concat(final_result)
    return out

In [561]:
search_string = input('please enter a valid search string \n')
result = final_result(search_string)
result = result.reset_index(drop=True)

please enter a valid search string 
Lipitor targets


# column mapping
####### '@_fa' 
####### 'link'
#######  'prism:url'
#######  'dc:identifier' : scopus id
#######  'eid'
#######  'dc:title' : title
#######  'dc:creator' : first_author
#######  'prism:publicationName' : journal
#######  'prism:issn'
#######  'prism:eIssn'
#######  'prism:volume'
#######  'prism:pageRange'
#######  'prism:coverDate'
#######  'prism:coverDisplayDate'
#######  'prism:doi' : doi
#######  'pii' : something to pull paper from
#######  'citedby-count'
#######  'affiliation'
#######  'pubmed-id'
#######  'prism:aggregationType'
#######  'subtype'
#######  'subtypeDescription'
#######  'article-number'
#######  'source-id'
#######  'openaccess' : open access = 1
#######  'openaccessFlag'
#######  'prism:issueIdentifier'
#######  'prism:isbn'

In [617]:
result

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:issn,prism:eIssn,...,pubmed-id,prism:aggregationType,subtype,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,prism:issueIdentifier,prism:isbn
0,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85083332824,2-s2.0-85083332824,Indirect photodegradation of fludioxonil by hy...,Yang J.,Ecotoxicology and Environmental Safety,01476513,10902414,...,32325330,Journal,ar,Article,110644,25076,0,False,NaN,NaN
1,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85078805182,2-s2.0-85078805182,Effects of Green Tea Extract on Atorvastatin P...,Abdelkawy K.S.,European Journal of Drug Metabolism and Pharma...,03787966,21070180,...,31997084,Journal,ar,Article,NaN,21328,0,False,3,NaN
2,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85084693940,2-s2.0-85084693940,Degradation versus inhibition: Development of ...,Li M.X.,Journal of Medicinal Chemistry,00222623,15204804,...,32321253,Journal,ar,Article,NaN,23041,0,False,9,NaN
3,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85085078179,2-s2.0-85085078179,Strengthening effects of bone marrow mononucle...,Yang Y.J.,Open Heart,2398595X,20533624,...,NaN,Journal,ar,Article,e001139,21100464778,1,True,1,NaN
4,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85083499125,2-s2.0-85083499125,Atorvastatin Reduces First and Subsequent Vasc...,Szarek M.,Journal of the American College of Cardiology,07351097,15583597,...,32194196,Journal,ar,Article,NaN,22401,0,False,17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,0031928765,2-s2.0-0031928765,Efficacy and safety of Stitivastatin 80 mg/day...,Stein E.A.,American Journal of Cardiology,00029149,NaN,...,9708659,Journal,ar,Article,NaN,22415,0,False,3,NaN
22,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,0031663613,2-s2.0-0031663613,Atorvastatin calcium: A new HMG-CoA reductase ...,Leiter L.,Canadian Journal of Clinical Pharmacology,1198581X,NaN,...,NaN,Journal,ar,Article,NaN,20125,0,False,3,NaN
23,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,0031195278,2-s2.0-0031195278,"A multicenter, double-blind, one-year study co...",Dart A.,American Journal of Cardiology,00029149,NaN,...,9205017,Journal,ar,Article,NaN,22415,0,False,1,NaN
24,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,18544412108,2-s2.0-18544412108,Efficacy and safety of atorvastatin compared t...,Bertolini S.,Atherosclerosis,00219150,NaN,...,9126664,Journal,ar,Article,NaN,22483,0,False,1-2,NaN


In [565]:
for scopus_id in result['dc:identifier']:
    

0     85083332824
1     85078805182
2     85084693940
3     85085078179
4     85083499125
         ...     
21     0031928765
22     0031663613
23     0031195278
24    18544412108
0      1842339933
Name: dc:identifier, Length: 701, dtype: object

In [614]:
b['14965203'].headers

{'allow': 'GET', 'Content-Encoding': 'gzip', 'Content-Type': 'text/xml;charset=UTF-8', 'Date': 'Fri, 10 Jul 2020 17:49:58 GMT', 'Last-Modified': 'Wed, 28 Mar 2018 16:41:29 GMT', 'Server': 'Apache-Coyote/1.1', 'vary': 'Origin', 'WARNING': 'Unauthorized request results in minimized metadata response.', 'X-ELS-APIKey': '7968ea68ad28c4627d768d46292800fe', 'X-ELS-ReqId': '0c051e8f-f0d6-4837-ad2f-cf961bcd8458', 'X-ELS-ResourceVersion': 'default', 'X-ELS-Status': 'WARNING - Unauthorized request results in minimized metadata response.', 'X-ELS-TransId': '89fb0775-f4b7-4d1f-8562-d5c62b120b08', 'Content-Length': '701', 'Connection': 'keep-alive'}

In [652]:
requests.get("https://www.sciencedirect.com/science/article/pii/S1931312818303779") 

<Response [403]>

In [624]:
for i in result['pubmed-id'].dropna().head(20):
    print(i)

32325330
31997084
32321253
32194196
31871222
32382557
31470153
31685825
31402538
31350867
31486641
31030372
31073648
31236614
31125254
31202119
31308572
30851734
30269353
30628690


In [733]:
search_string = 'gwas AND \'natural language processing\''
a = get_summary(search_string,apikey)
a

{'total_results': 295,
 'search_terms': "gwas AND 'natural language processing'",
 'link_used': "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=gwas%20AND%20'natural%20language%20processing'&apiKey=222a2a95a093a5c6049c145c09ed85a9",
 'apikey': '222a2a95a093a5c6049c145c09ed85a9'}

In [734]:
b = final_result(search_string)
b = b.reset_index(drop=True)

https://api.elsevier.com/content/search/scopus?start=300&count=25&query=gwas%20AND%20'natural%20language%20processing'&apiKey=222a2a95a093a5c6049c145c09ed85a9 'entry'


In [735]:
b

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,...,prism:aggregationType,subtype,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,prism:issn,pii,prism:isbn
0,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85077697294,2-s2.0-85077697294,Genome-wide association and Mendelian randomis...,Shah S.,Nature Communications,20411723,11,...,Journal,ar,Article,163,19700182758,1,True,NaN,NaN,NaN
1,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85083807817,2-s2.0-85083807817,Genome-wide DNA methylation analysis of heavy ...,Osborne A.J.,Translational Psychiatry,21583188,10,...,Journal,ar,Article,114,19900191866,1,True,NaN,NaN,NaN
2,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85082148270,2-s2.0-85082148270,ENIGMA and global neuroscience: A decade of la...,Thompson P.M.,Translational Psychiatry,21583188,10,...,Journal,re,Review,100,19900191866,1,True,NaN,NaN,NaN
3,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85082076222,2-s2.0-85082076222,An analytic framework for exploring sampling a...,Beesley L.J.,Statistics in Medicine,10970258,39,...,Journal,ar,Article,NaN,20086,0,False,02776715,NaN,NaN
4,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,85086694398,2-s2.0-85086694398,PyMeSHSim: An integrative python package for b...,Luo Z.H.,BMC Bioinformatics,14712105,21,...,Journal,ar,Article,252,17929,1,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,77958597919,2-s2.0-77958597919,Leveraging informatics for genetic studies: Us...,Kullo I.J.,Journal of the American Medical Informatics As...,NaN,17,...,Journal,ar,Article,NaN,23600,1,True,10675027,NaN,NaN
291,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,84885558608,2-s2.0-84885558608,Knowledge-Driven Approaches to Genome-Scale An...,Tipney H.,Knowledge-Based Bioinformatics: From Analysis ...,NaN,NaN,...,Book,ch,Chapter,NaN,21100261689,0,False,NaN,NaN,"[{'@_fa': 'true', '$': '9780470748312'}]"
292,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,77952822074,2-s2.0-77952822074,PheWAS: Demonstrating the feasibility of a phe...,Denny J.C.,Bioinformatics,14602059,26,...,Journal,ar,Article,btq126,17945,1,True,13674803,NaN,NaN
293,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,70349885499,2-s2.0-70349885499,An algorithm for learning maximum entropy prob...,Miller D.J.,Bioinformatics,14602059,25,...,Journal,ar,Article,NaN,17945,1,True,13674803,NaN,NaN


In [738]:
d = b['pii'].dropna()
len(d)

42

In [739]:
c = b['prism:doi'].dropna()
len(c)

284

In [747]:
f = {}
for i in d:
    data = FullDoc(sd_pii = i)
    if data.read(client):
        f[i] = 'Paper found'
    else:
        f[i] = 'No full text article found'

In [748]:
e = {}
for i in c:
    data = FullDoc(doi = i)
    if data.read(client):
        e[i] = 'paper found'
    else:
        e[i] = 'No full text article found'

In [752]:
col = 0
for i in e.values():
    if i == 'No full text article found':
        col += 1
print(295-col)

54


In [731]:
len(d)

42

In [749]:
e

{'10.1038/s41467-019-13690-5': 'No full text article found',
 '10.1038/s41398-020-0800-3': 'No full text article found',
 '10.1038/s41398-020-0705-1': 'No full text article found',
 '10.1002/sim.8524': 'No full text article found',
 '10.1186/s12859-020-03583-6': 'No full text article found',
 '10.3390/biom10060915': 'No full text article found',
 '10.3390/cancers12051245': 'No full text article found',
 '10.1007/s10654-020-00640-5': 'No full text article found',
 '10.1098/rsif.2019.0610': 'No full text article found',
 '10.1016/j.bbr.2020.112521': 'paper found',
 '10.3389/fgene.2020.00350': 'No full text article found',
 '10.1017/neu.2019.41': 'No full text article found',
 '10.1534/g3.119.400884': 'No full text article found',
 '10.1016/j.ejmg.2019.103828': 'paper found',
 '10.1186/s12967-020-02312-0': 'No full text article found',
 '10.1002/sim.8445': 'No full text article found',
 '10.1016/j.pnpbp.2019.109764': 'paper found',
 '10.1093/bioinformatics/btz767': 'No full text article f

In [750]:
f

{'S0166432819317942': 'Paper found',
 'S0924270819000413': 'No full text article found',
 'S1769721219306500': 'Paper found',
 'S0278584619304282': 'Paper found',
 'S0888754318307419': 'Paper found',
 'S0223523419307366': 'Paper found',
 'S1532046419301923': 'Paper found',
 'S1566253518304482': 'Paper found',
 'S092144881830926X': 'Paper found',
 'S0168945219304066': 'Paper found',
 'S0168952519300915': 'Paper found',
 'S0033291719000151': 'No full text article found',
 'S0925443918303326': 'Paper found',
 'S0896627319302855': 'Paper found',
 'S0163725818301906': 'Paper found',
 'S2468501118300191': 'Paper found',
 'S030105111830200X': 'Paper found',
 'S1532046418301795': 'Paper found',
 'S1931312818303779': 'Paper found',
 'S2212877818301984': 'Paper found',
 'S1084952117303828': 'Paper found',
 'B9780128018125000093': 'Paper found',
 'S1876162318300312': 'Paper found',
 'S187711731830019X': 'Paper found',
 'S0160289616302367': 'Paper found',
 'S0957417416307175': 'Paper found',
 'S00

In [757]:
b[['pii','prism:doi']]

,pii,prism:doi
0,NaN,10.1038/s41467-019-13690-5
1,NaN,10.1038/s41398-020-0800-3
2,NaN,10.1038/s41398-020-0705-1
3,NaN,10.1002/sim.8524
4,NaN,10.1186/s12859-020-03583-6
...,...,...
290,NaN,10.1136/jamia.2010.004366
291,NaN,10.1002/9780470669716.ch2
292,NaN,10.1093/bioinformatics/btq126
293,NaN,10.1093/bioinformatics/btp435


In [763]:
e = pd.DataFrame.from_dict(e,orient = 'index' ,columns = ['paper']).reset_index()

In [762]:
f = pd.DataFrame.from_dict(f,orient = 'index' ,columns = ['paper']).reset_index()

In [ ]:
b[['pii','prism:doi']].merge(e, )

In [764]:
e

,index,paper
0,10.1038/s41467-019-13690-5,No full text article found
1,10.1038/s41398-020-0800-3,No full text article found
2,10.1038/s41398-020-0705-1,No full text article found
3,10.1002/sim.8524,No full text article found
4,10.1186/s12859-020-03583-6,No full text article found
...,...,...
278,10.1136/jamia.2010.004366,No full text article found
279,10.1002/9780470669716.ch2,No full text article found
280,10.1093/bioinformatics/btq126,No full text article found
281,10.1093/bioinformatics/btp435,No full text article found
